In [1]:
import torch
import json
import pandas as pd
import numpy as np

torch.__version__

'1.12.1+cu113'

In [2]:
torch.cuda.is_available()

True

In [5]:
FILE = 'arxiv-metadata-oai-snapshot.json'

def get_data():
    with open(FILE) as f:
        for line in f:
            yield line

### TODO: dataframe에 JSON LOAD Columns 모두 추가해서 csv 저장

In [6]:
dataframe = {
    'title': [],
    'year': [],
    'abstract': []
}

data = get_data()
for i, paper in enumerate(data):
    paper = json.loads(paper)
    # print(paper)
    # break
    try:
        date = int(paper['update_date'].split('-')[0])
        if date > 2019:
            dataframe['title'].append(paper['title'])
            dataframe['year'].append(date)
            dataframe['abstract'].append(paper['abstract'])
    except: pass

데이터 너무 많음 -> cs 관련?
인용수 -> 적은 애들 neg sample로 사용 가능

graph -> 인접행렬

In [7]:
df = pd.DataFrame(dataframe)

In [8]:
del dataframe

In [8]:
df

,title,year,abstract
0,Convergence of the discrete dipole approximati...,2022,We performed a rigorous theoretical converge...
1,The discrete dipole approximation: an overview...,2022,We present a review of the discrete dipole a...
2,Mapping radii of metric spaces,2021,It is known that every closed curve of lengt...
3,What can emission lines tell us?,2023,1 Generalities\n 2 Empirical diagnostics ba...
4,The affine part of the Picard scheme,2021,We describe the maximal torus and maximal un...
...,...,...,...
886395,The Hamiltonian structure of the dispersionles...,2020,The Hamiltonian structure of the two-dimensi...
886396,DNA Transcription Mechanism with a Moving Enzyme,2021,Previous numerical investigations of an one-...
886397,Degenerate Frobenius manifolds and the bi-Hami...,2020,The bi-Hamiltonian structure of certain mult...
886398,Hypercomplex Integrable Systems,2020,In this paper we study hypercomplex manifold...


In [9]:
max([len(i) for i in df.abstract])

5089

In [10]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

loading existing colbert_linear and sparse_linear---------


In [11]:
# 두 시간 걸리니 참고만 하셈
output = model.encode(list(df.abstract),
                      batch_size=12, 
                      max_length=5090, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                      )['dense_vecs']

Inference Embeddings: 100%|██████████| 73867/73867 [1:57:03<00:00, 10.52it/s]  


In [12]:
emb_df = pd.DataFrame(output)

In [13]:
emb_df.to_csv('embedding.csv', index=False)

# Inference

In [2]:
output = pd.read_csv('embedding.csv').to_numpy()

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [9]:
y = df.index

In [10]:
nn = KNeighborsClassifier(n_neighbors=6)
nn.fit(output, y)

KNeighborsClassifier(n_neighbors=6)

In [11]:
import random

맨처음 시나리오 있으면 좋겠다.

In [12]:
for _ in range(5):
    idx = random.randint(0, len(y))
    sample = df["title"][idx]
    dist, index = nn.kneighbors(X=output[idx,:].reshape(1,-1))
    print(f"Sample:\n{sample}\n")
    for i in range(1,6):
        print(f"Recommendation {i}:\n{df['title'][index[0][i]]}\n")
    print("===============\n")

Sample:
Machine learning Majorana nanowire disorder landscape

Recommendation 1:
Density of states, transport, and topology in disordered Majorana
  nanowires

Recommendation 2:
Deep neural networks for inverse problems in mesoscopic physics:
  Characterization of the disorder configuration from quantum transport
  properties

Recommendation 3:
Ballistic Majorana nanowire devices

Recommendation 4:
Crossover between trivial zero modes in Majorana nanowires

Recommendation 5:
Decoding the DC and optical conductivities of disordered MoS$_{2}$
  films: an inverse problem


Sample:
Geodesic completeness of the $H^{3/2}$ metric on $\mathrm{Diff}(S^{1})$

Recommendation 1:
Completeness and geodesic distance properties for fractional Sobolev
  metrics on spaces of immersed curves

Recommendation 2:
Fractional Sobolev metrics on spaces of immersions

Recommendation 3:
An Example of Banach and Hilbert manifold: the universal Teichm\"uller
  space

Recommendation 4:
Semi-invariant Riemannian met